# Импорт библиотек

In [ ]:
!pip install comet-ml

import comet_ml
from comet_ml import Experiment

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install pytorch_lightning==1.9.4

In [ ]:
import pytorch_lightning as pl

logger = pl.loggers.comet.CometLogger(
    # secret logs :)
)

In [ ]:
!pip install nemo_toolkit[all]

In [ ]:
import nemo
import nemo.collections.asr as nemo_asr

[NeMo W 2023-04-17 06:33:34 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-04-17 06:33:41 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [ ]:
BRANCH = 'r1.0.0b3'

!pip install pydub
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install omegaconf
!pip install torchaudio>=0.6.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import os
import json
from google.colab import drive
from omegaconf import OmegaConf

import wave
import torch
import librosa
import torchaudio
import librosa.display
import soundfile as sf
from scipy import signal
import IPython.display as ipd
from pydub import AudioSegment
import torchaudio.functional as F 
from torchaudio.utils import download_asset

import numpy as np
import pandas as pd

import shutil
from google.colab import files

# Загрузка файлов

In [ ]:
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [ ]:
!mkdir '/content/dataset'
!mkdir '/content/dataset/train'
!mkdir '/content/dataset/test'
!mkdir '/content/dataset/dev'

!mkdir '/content/dataset/far'

In [ ]:
!unzip '/content/gdrive/MyDrive/Colab Notebooks/neymark/train.zip' -d '/content/dataset/'
!tar -xvf '/content/gdrive/MyDrive/Colab Notebooks/neymark/v0.02_validation.tar' -C '/content/dataset/dev'
!tar -xvf '/content/gdrive/MyDrive/Colab Notebooks/neymark/v0.02_test.tar' -C '/content/dataset/test'

!unzip '/content/gdrive/MyDrive/Colab Notebooks/neymark/farph.zip' -d '/content/dataset/far'

# Создание .json файлов для хранения путей к аудиозаписям

In [ ]:
import wave

def extract_paths(file_path, outpath):
    labels = ['marvin', 'unknow']
    out = []

    for root, dirs, files in os.walk(file_path):
        for i in dirs:
            for root2, dirs2, files2 in os.walk(file_path+i):
                for j in files2:
                    path = file_path + i + '/' + j

                    duration = 0.0
                    with wave.open(path) as mywav:
                            duration = mywav.getnframes() / mywav.getframerate()

                    command = labels[0] if i == labels[0] else labels[1]
                    
                    out.append({"audio_filepath": path, "duration": duration, 
                                "command": command})


    with open(outpath, 'w') as f:
        for val in out:
            json.dump(val, f)
            f.write("\n")
            f.flush()

In [ ]:
extract_paths('/content/dataset/dev/', '/content/dev.json')
extract_paths('/content/dataset/test/', '/content/test.json')
extract_paths('/content/dataset/train/', '/content/train.json')

extract_paths('/content/dataset/far/', '/content/far.json')

In [ ]:
!mkdir jsons_files
!mv dev.json jsons_files
!mv test.json jsons_files
!mv train.json jsons_files

In [ ]:
!mkdir /content/jsons_files_train_augm

In [ ]:
shutil.make_archive("/content/jsons_files_train_augm", "zip", "/content/jsons_files_train_augm")
!cp /content/jsons_files_train_augm.zip "/content/gdrive/MyDrive/Colab Notebooks/neymark"
files.download('/content/jsons_files_train_augm.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Скачиваем yaml файл с гитхаба и меняем его под наш датасет

In [ ]:
MODEL_CONFIG = "matchboxnet_3x1x64_v2.yaml"

if not os.path.exists(f"configs/{MODEL_CONFIG}"):
  !wget -P configs/ "https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/{MODEL_CONFIG}"

# Создаем переменные, хранящие пути к файлам

In [ ]:
train_dataset = os.path.join('/content/train.json')
val_dataset = os.path.join('/content/dev.json')
test_dataset = os.path.join('/content/test.json')

In [ ]:
far_dataset = os.path.join('/content/far.json')

In [ ]:
!head -n 5 {train_dataset}

{"audio_filepath": "/content/dataset/train/off/72242187_nohash_3.wav", "duration": 1.0, "command": "unknow"}
{"audio_filepath": "/content/dataset/train/off/82e7204d_nohash_2.wav", "duration": 1.0, "command": "unknow"}
{"audio_filepath": "/content/dataset/train/off/f7879738_nohash_0.wav", "duration": 1.0, "command": "unknow"}
{"audio_filepath": "/content/dataset/train/off/de89e2ca_nohash_5.wav", "duration": 1.0, "command": "unknow"}
{"audio_filepath": "/content/dataset/train/off/25132942_nohash_2.wav", "duration": 1.0, "command": "unknow"}


# Конфигурация модели

In [ ]:
config_path = f"{MODEL_CONFIG}"
config = OmegaConf.load(config_path)
print(OmegaConf.to_yaml(config))

name: MatchboxNet-3x1x64-v2
model:
  sample_rate: 22050
  timesteps: 128
  repeat: 3
  dropout: 0.2
  kernel_size_factor: 1.0
  labels_full:
  - marvin
  - unknow
  labels: ${model.labels_full}
  train_ds:
    manifest_filepath: ???
    sample_rate: ${model.sample_rate}
    labels: ${model.labels}
    batch_size: 128
    shuffle: true
  validation_ds:
    manifest_filepath: ???
    sample_rate: ${model.sample_rate}
    labels: ${model.labels}
    batch_size: 128
    shuffle: false
    val_loss_idx: 0
  test_ds:
    manifest_filepath: null
    sample_rate: ${model.sample_rate}
    labels: ${model.labels}
    batch_size: 128
    shuffle: false
    test_loss_idx: 0
  preprocessor:
    cls: nemo.collections.asr.modules.AudioToMFCCPreprocessor
    params:
      window_size: 0.025
      window_stride: 0.01
      window: hann
      n_mels: 64
      n_mfcc: 64
      n_fft: 512
  spec_augment:
    cls: nemo.collections.asr.modules.SpectrogramAugmentation
    params:
      freq_masks: 2
      ti

In [ ]:
labels = config.model.labels
sample_rate = 22050

In [ ]:
config.model.train_ds.manifest_filepath = train_dataset
config.model.validation_ds.manifest_filepath = val_dataset
config.model.test_ds.manifest_filepath = test_dataset

# Подготовка к обучению

In [ ]:
cuda = 1 if torch.cuda.is_available() else 0

config.trainer.max_epochs = 20
config.trainer.accelerator = 'gpu'

In [ ]:
conf = {'max_epochs': 20, 'num_nodes': 1, 'accelerator': 'gpu', 
        'accumulate_grad_batches': 1, 
        'log_every_n_steps': 1, 'val_check_interval': 0.25, 'logger': logger}

In [ ]:
trainer = pl.Trainer(**conf)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
asr_model = nemo_asr.models.EncDecClassificationModel(cfg=config.model, trainer=trainer)

[NeMo I 2023-04-10 16:06:59 collections:298] Filtered duration for loading collection is 0.000000.
[NeMo I 2023-04-10 16:06:59 collections:301] Dataset loaded with 166930 items, total duration of  46.19 hours.
[NeMo I 2023-04-10 16:06:59 collections:303] # 166930 files loaded accounting to # 2 labels
[NeMo I 2023-04-10 16:06:59 collections:298] Filtered duration for loading collection is 0.000000.
[NeMo I 2023-04-10 16:06:59 collections:301] Dataset loaded with 9982 items, total duration of  2.74 hours.
[NeMo I 2023-04-10 16:06:59 collections:303] # 9982 files loaded accounting to # 2 labels
[NeMo I 2023-04-10 16:06:59 collections:298] Filtered duration for loading collection is 0.000000.
[NeMo I 2023-04-10 16:06:59 collections:301] Dataset loaded with 4890 items, total duration of  1.36 hours.
[NeMo I 2023-04-10 16:06:59 collections:303] # 4890 files loaded accounting to # 2 labels


# Обучение

In [ ]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

In [ ]:
# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

In [ ]:
trainer.fit(asr_model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2023-04-10 16:14:37 modelPT:722] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.95, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.05
        weight_decay: 0.001
    )
[NeMo I 2023-04-10 16:14:37 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.PolynomialHoldDecayAnnealing object at 0x7fc09f4f09d0>" 
    will be used during training (effective maximum steps = 26100) - 
    Parameters : 
    (power: 2.0
    warmup_ratio: 0.05
    hold_ratio: 0.45
    min_lr: 0.001
    last_epoch: -1
    max_steps: 26100
    )


INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type                             | Params
-----------------------------------------------------------------------
0 | spec_augmentation | SpectrogramAugmentation          | 0     
1 | crop_or_pad       | CropOrPadSpectrogramAugmentation | 0     
2 | preprocessor      | AudioToMFCCPreprocessor          | 0     
3 | encoder           | ConvASREncoder                   | 104 K 
4 | decoder           | ConvASRDecoderClassification     | 258   
5 | loss              | CrossEntropyLoss                 | 0     
6 | _accuracy         | TopKClassificationAccuracy       | 0     
-----------------------------------------------------------------------
104 K     Trainable params
0         Non-trainable params
104 K     Total params
0.419     Total estimated model params size (MB)
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/annanet/asr-keyword-spotting/c2713e48f198482fb3b77242379d3cf3
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     global_step [26100]                   : (0.0, 26099.0)
COMET INFO:     learning_rate [26100]                 : (3.828483750112355e-05, 0.05000000074505806)
COMET INFO:     loss [2610]                           : (0.0026562761049717665, 0.682393491268158)
COMET INFO:     train_loss [26100]                    : (0.0023350499104708433, 0.6973329782485962)

## Смотрим на тестовую выборку

In [ ]:
trainer.test(asr_model, ckpt_path=None)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/annanet/asr-keyword-spotting/c2713e48f198482fb3b77242379d3cf3



Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     test_epoch_top@1      │    0.9985685348510742     │
│         test_loss         │   0.005626469384878874    │
└───────────────────────────┴───────────────────────────┘

COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/annanet/asr-keyword-spotting/c2713e48f198482fb3b77242379d3cf3
COMET INFO:   Metrics:
COMET INFO:     test_epoch_top@1 : 0.9985685348510742
COMET INFO:     test_loss        : 0.005626469384878874
COMET INFO:   Others:
COMET INFO:     Created from : pytorch-lightning
COMET INFO:     Name         : train_augmented_only
COMET INFO:   Parameters:
COMET INFO:     cfg/crop_or_pad_augment/_target_     : nemo.collections.asr.modules.CropOrPadSpectrogramAugmentation
COMET INFO:     cfg/crop_or_pad_augment/audio_length : 128
COMET INFO:     cfg/decoder/_target_         

# Результаты train, dev, test

In [ ]:
config.model.test_ds.sample_rate

22050

In [ ]:
restored_model = nemo_asr.models.EncDecClassificationModel.load_from_checkpoint('/content/epoch=19-step=26099.ckpt')
restored_model.setup_multiple_test_data({'manifest_filepath': os.path.join('/content/test.json'), 'sample_rate': 22050, 'labels': ['marvin', 'unknow'], 'batch_size': 128, 'shuffle': False})
restored_model.setup_multiple_validation_data({'manifest_filepath': os.path.join('/content/dev.json'), 'sample_rate': 22050, 'labels': ['marvin', 'unknow'], 'batch_size': 128, 'shuffle': False})
restored_model.setup_training_data({'manifest_filepath': os.path.join('/content/train.json'), 'sample_rate': 22050, 'labels': ['marvin', 'unknow'], 'batch_size': 128, 'shuffle': False})

In [ ]:
test_dl = restored_model._test_dl
dev_dl = restored_model._validation_dl
train_dl = restored_model._train_dl

In [ ]:
cpu_model = restored_model.cpu()
cpu_model.eval()

In [ ]:
@torch.no_grad()
def extract_logits(model, dataloader):
  logits_buffer = []
  label_buffer = []

  # Follow the above definition of the test_step
  for batch in dataloader:
    audio_signal, audio_signal_len, labels, labels_len = batch
    logits = model(input_signal=audio_signal, input_signal_length=audio_signal_len)

    logits_buffer.append(logits)
    label_buffer.append(labels)
    print(".", end='')
  print()
  
  print("Finished extracting logits !")
  logits = torch.cat(logits_buffer, 0)
  labels = torch.cat(label_buffer, 0)
  return logits, labels


In [ ]:
def get_labels_logits(model, dl):
    logits, labels = extract_logits(model, dl)
    print("Logits:", logits.shape, "Labels :", labels.shape)

    return logits, labels

In [ ]:
clogits_train, labels_train = get_labels_logits(cpu_model, train_dl)

logits_test, labels_test = get_labels_logits(cpu_model, test_dl)

logits_dev, labels_dev = get_labels_logits(cpu_model, dev_dl)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
acc = cpu_model._accuracy(logits=logits_train, labels=labels_train)
print("Accuracy : ", float(acc[0]*100))

acc = cpu_model._accuracy(logits=logits_dev, labels=labels_dev)
print("Accuracy : ", float(acc[0]*100))

acc = cpu_model._accuracy(logits=logits_test, labels=labels_test)
print("Accuracy : ", float(acc[0]*100))

Accuracy :  99.60282897949219
Accuracy :  99.50911712646484
Accuracy :  99.8772964477539


## Метрики

In [ ]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score

In [ ]:
probs_test = torch.softmax(logits_test, dim=-1)
probas_test, preds_test = torch.max(probs_test, dim=-1)

print(balanced_accuracy_score(labels_test, preds_test))

probs_train = torch.softmax(logits_train, dim=-1)
probas_train, preds_train = torch.max(probs_train, dim=-1)

print(balanced_accuracy_score(labels_train, preds_train))

probs_dev = torch.softmax(logits_dev, dim=-1)
probas_dev, preds_dev = torch.max(probs_dev, dim=-1)

print(balanced_accuracy_score(labels_dev, preds_dev))

0.9086809890253219
0.9960282753249865
0.8793850031307884


In [ ]:
print(f1_score(labels_test, preds_test))
print(f1_score(labels_train, preds_train))
print(f1_score(labels_dev, preds_dev))

0.9993849938499385
0.9960432792442245
0.9975024211223813


# Посмотрим на неверно классифицированные данные

In [ ]:
# для вывода labels
class ReverseMapLabel:
    def __init__(self, data_loader):
        self.label2id = dict(data_loader.dataset.label2id)
        self.id2label = dict(data_loader.dataset.id2label)

    def __call__(self, pred_idx, label_idx):
        return self.id2label[pred_idx], self.id2label[label_idx]

In [ ]:
def indices_inc_lables(dl, logits, labels, cpu_model):
    sample_idx = 0
    incorrect_preds = []
    rev_map = ReverseMapLabel(dl)

    probs = torch.softmax(logits, dim=-1)
    probas, preds = torch.max(probs, dim=-1)

    total_count = cpu_model._accuracy.total_counts_k[0]
    incorrect_ids = (preds != labels).nonzero()
    for idx in incorrect_ids:
        proba = float(probas[idx][0])
        pred = int(preds[idx][0])
        label = int(labels[idx][0])
        idx = int(idx[0]) + sample_idx

        incorrect_preds.append((idx, *rev_map(pred, label), proba))

    print(f"Num test samples : {total_count.item()}")
    print(f"Num errors : {len(incorrect_preds)}")

    incorrect_preds = sorted(incorrect_preds, key=lambda x: x[-1], reverse=False)

    return incorrect_preds

In [ ]:
# train тут напутано с Num test samples, должно быть 166930
incorrect_preds_train = indices_inc_lables(train_dl, logits_train, labels_train, cpu_model)

Num test samples : 4890
Num errors : 663


In [ ]:
#dev тут напутано с Num test samples, должно быть 19692
incorrect_preds_dev = indices_inc_lables(dev_dl, logits_dev, labels_dev, cpu_model)

Num test samples : 4890
Num errors : 49


In [ ]:
#test
incorrect_preds_test = indices_inc_lables(test_dl, logits_test, labels_test, cpu_model)

Num test samples : 4890
Num errors : 6


In [ ]:
86 / 7198 * 60 * 60

43.01194776326757

### Списки неверно классифицированных

In [ ]:
for incorrect_sample in incorrect_preds_test:
    print(str(incorrect_sample))

(1554, 'marvin', 'unknow', 0.5794825553894043)
(4413, 'marvin', 'unknow', 0.6885301470756531)
(1581, 'marvin', 'unknow', 0.706212043762207)
(1098, 'marvin', 'unknow', 0.9239190816879272)
(3899, 'unknow', 'marvin', 0.9893389344215393)
(3896, 'unknow', 'marvin', 0.9970242381095886)


In [ ]:
for incorrect_sample in incorrect_preds_dev:
    print(str(incorrect_sample))

(2172, 'marvin', 'unknow', 0.5208429098129272)
(3761, 'unknow', 'marvin', 0.5593491196632385)
(3671, 'unknow', 'marvin', 0.6029666066169739)
(3739, 'unknow', 'marvin', 0.6206477284431458)
(4273, 'marvin', 'unknow', 0.6211844682693481)
(3759, 'unknow', 'marvin', 0.6567263007164001)
(8557, 'marvin', 'unknow', 0.6877593994140625)
(3691, 'unknow', 'marvin', 0.6949358582496643)
(3785, 'unknow', 'marvin', 0.7551493048667908)
(3833, 'unknow', 'marvin', 0.7611287236213684)
(3735, 'unknow', 'marvin', 0.8598874807357788)
(3718, 'unknow', 'marvin', 0.8643510341644287)
(3676, 'unknow', 'marvin', 0.866972804069519)
(3727, 'unknow', 'marvin', 0.8754906058311462)
(3654, 'unknow', 'marvin', 0.8771348595619202)
(3644, 'unknow', 'marvin', 0.8805410265922546)
(3827, 'unknow', 'marvin', 0.8942705988883972)
(3819, 'unknow', 'marvin', 0.9051885008811951)
(3792, 'unknow', 'marvin', 0.9170592427253723)
(3686, 'unknow', 'marvin', 0.9296338558197021)
(3763, 'unknow', 'marvin', 0.9381946921348572)
(3743, 'unknow

In [ ]:
for incorrect_sample in incorrect_preds: #train
    print(str(incorrect_sample))

(152540, 'unknow', 'marvin', 0.5017963647842407)
(118719, 'unknow', 'marvin', 0.5028939843177795)
(59767, 'unknow', 'marvin', 0.5031280517578125)
(93231, 'unknow', 'marvin', 0.5031501054763794)
(161683, 'unknow', 'marvin', 0.5045109391212463)
(111069, 'unknow', 'marvin', 0.5051589012145996)
(110956, 'unknow', 'marvin', 0.5054197907447815)
(137889, 'unknow', 'marvin', 0.5067870616912842)
(130048, 'marvin', 'unknow', 0.5106440186500549)
(42049, 'unknow', 'marvin', 0.5109167098999023)
(82124, 'unknow', 'marvin', 0.5115885138511658)
(148922, 'unknow', 'marvin', 0.5156765580177307)
(61291, 'unknow', 'marvin', 0.5162248611450195)
(60909, 'unknow', 'marvin', 0.5171166658401489)
(137321, 'unknow', 'marvin', 0.5190425515174866)
(50803, 'unknow', 'marvin', 0.5207676887512207)
(54064, 'unknow', 'marvin', 0.521946132183075)
(136233, 'unknow', 'marvin', 0.5219788551330566)
(136053, 'unknow', 'marvin', 0.5236688852310181)
(161294, 'unknow', 'marvin', 0.5239557027816772)
(139532, 'unknow', 'marvin', 

# FAR per hour

In [ ]:
restored_model.setup_multiple_test_data({'manifest_filepath': os.path.join('/content/far.json'), 'sample_rate': 22050, 'labels': ['marvin', 'unknow'], 'batch_size': 128, 'shuffle': False})
far_dl = restored_model._test_dl

logits_far, labels_far = get_labels_logits(cpu_model, far_dl)

.........................................................
Finished extracting logits !
Logits: torch.Size([7198, 2]) Labels : torch.Size([7198])


In [ ]:
acc = cpu_model._accuracy(logits=logits_far, labels=labels_far)
print("Accuracy : ", float(acc[0]*100))

Accuracy :  98.80522155761719


In [ ]:
#far
incorrect_preds_far = indices_inc_lables(far_dl, logits_far, labels_far, cpu_model)

Num test samples : 7198
Num errors : 86


In [ ]:
for incorrect_sample in incorrect_preds:
    print(str(incorrect_sample)) #FAR

(4289, 'marvin', 'unknow', 0.504155695438385)
(5578, 'marvin', 'unknow', 0.5095963478088379)
(83, 'marvin', 'unknow', 0.5136798024177551)
(2052, 'marvin', 'unknow', 0.5145758390426636)
(4814, 'marvin', 'unknow', 0.5188309550285339)
(1728, 'marvin', 'unknow', 0.5189707279205322)
(3480, 'marvin', 'unknow', 0.5313900113105774)
(2336, 'marvin', 'unknow', 0.535024106502533)
(204, 'marvin', 'unknow', 0.5356135368347168)
(235, 'marvin', 'unknow', 0.5432113409042358)
(4933, 'marvin', 'unknow', 0.5681405067443848)
(6107, 'marvin', 'unknow', 0.576557993888855)
(2057, 'marvin', 'unknow', 0.5776137113571167)
(599, 'marvin', 'unknow', 0.5778594613075256)
(227, 'marvin', 'unknow', 0.5802414417266846)
(5861, 'marvin', 'unknow', 0.5820289254188538)
(589, 'marvin', 'unknow', 0.5907805562019348)
(1449, 'marvin', 'unknow', 0.592566728591919)
(4395, 'marvin', 'unknow', 0.5961357355117798)
(6727, 'marvin', 'unknow', 0.6095893979072571)
(5114, 'marvin', 'unknow', 0.6132431626319885)
(1745, 'marvin', 'unknow